In [55]:
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnableLambda, RunnableBranch, RunnableSequence
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq
from pydantic import BaseModel, Field
from typing import Literal
from langchain_core.output_parsers import PydanticOutputParser


groq_api_key=os.getenv("GROQ_API_KEY")

model=ChatGroq(groq_api_key=groq_api_key,model_name="qwen-2.5-32b")

In [56]:
class StuOutputCorrectness(BaseModel):
    sentiment: Literal['correct', 'partially_correct', 'incorrect'] = Field(description='Give the sentiment from the student response')
    
parser = PydanticOutputParser(pydantic_object=StuOutputCorrectness)

In [57]:
system_messages = "You are a helpful assistant"

In [58]:
from langchain.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_messages),
        ("human", 
         "Analyze the following student response and classify its correctness. "
         "Determine whether it is 'correct', 'partially_correct', or 'incorrect'.\n\n"
         "Student Response: {response}\n\n"
         "Provide the classification following this format:\n{format_instruction}")
    ]
).partial(format_instruction=parser.get_format_instructions())

In [59]:
classifier_chain = chat_prompt | model | parser

In [60]:
classifier_chain.invoke(
    {
        'response': "This sum of 2+2=5"
    }
)

StuOutputCorrectness(sentiment='incorrect')

## ***Adaptive Prompt With Messages History***

In [61]:
chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Analyze the following student response and classify its correctness."
         "Determine whether it is 'correct', 'partially_correct', or 'incorrect'.\n\n"
         "Student Response: {response}\n\n"
         "Provide the classification following this format:\n{format_instruction}"),
        MessagesPlaceholder(variable_name='messages')
    ]
).partial(format_instruction=parser.get_format_instructions())

In [62]:
classifier_chain = chat_prompt | model | parser

In [63]:
## List of messages
messages = [
                HumanMessage(content="solve this problem and 2+2=0?"),
                AIMessage(content="This is a addition problem and think about you have tow apple and your friend also give 2 more apple now how many apple you have. Its a hints to solve this problem."),
            ]

In [64]:
classifier_chain.invoke(
    {
        
        'messages': messages,
        'response':"maybe i have 3 apple or 4 i'm bit confused. so 2+2=4",
        
    }
)

StuOutputCorrectness(sentiment='partially_correct')

## Add Branch

In [92]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_correct = ChatPromptTemplate.from_messages(
    [
        ("system", 
         "Student gave the correct response and the response is {response}. "
         "Now give an adaptive prompt so the model can generate only the best response for the next step. "
         "Also analyze previous message history — it will help generate a better adaptive prompt."),
        MessagesPlaceholder(variable_name="messages")
    ]
)

prompt_partially_correct = ChatPromptTemplate.from_messages(
    [
        ("system", 
         """
         your task is generate a best adaptive prompt. And this adaptive prompt goes to my main model as the system prompt. Using this prompt the main model will be generated the elaborated response you just generate one the adaptive prompt only. Think you just help to main model as a instructor like a boss.
         for generate one best adaptive prompt you can analysis the previous chat and current response also.
         Student gave a response which is partially correct and the response is {response}. 
         """
         ),
        MessagesPlaceholder(variable_name="messages")
    ]
)

prompt_incorrect = ChatPromptTemplate.from_messages(
    [
        ("system", 
         "Student gave a totally incorrect response and the response is {response}. "
         "Analyze the student’s misunderstanding — maybe they need clarification on core concepts. "
         "Provide elaborated hints or ask guiding questions. "
         "Use previous message history to craft the best adaptive prompt for the next step."),
        MessagesPlaceholder(variable_name="messages")
    ]
)


In [93]:
str_parser = StrOutputParser()

In [94]:
branch_chain = RunnableBranch(
    (lambda x: x["sentiment"] == 'correct', prompt_correct | model | str_parser),
    (lambda x: x["sentiment"] == 'partially_correct', prompt_partially_correct | model | str_parser),
    (lambda x: x["sentiment"] == 'incorrect', prompt_incorrect | model | str_parser),
    RunnableLambda(lambda x: "Couldn't determine sentiment.")
)


In [95]:
from langchain_core.runnables import RunnableLambda

# Wrap classifier_chain to return full input + parsed sentiment
def enrich_with_sentiment(input: dict) -> dict:
    sentiment_result = classifier_chain.invoke(input)
    print(sentiment_result)
    return {
        **input,  # includes 'messages' and 'response'
        'sentiment': sentiment_result.sentiment
    }

enriched_chain = RunnableLambda(enrich_with_sentiment)


In [96]:
final_chain = enriched_chain | branch_chain

In [91]:
adaptive_prompt = final_chain.invoke(
    {
        
        'messages': messages,
        'response':"maybe i have 3 apple or 4 i'm bit confused. so 2+2=4",
        
    }
)
print(adaptive_prompt)

sentiment='partially_correct'
Based on the student's response and the problem at hand, here is an adaptive prompt to guide the main model in generating a more elaborated and helpful response:

---

**Adaptive Prompt:**

The student has provided a partially correct response, showing some confusion about the quantities involved. They are struggling with a basic addition problem and seem to be mixing up different scenarios. The student's response indicates a need for a clearer, step-by-step explanation and a concrete example to solidify their understanding.

You should confirm the correct answer for the addition problem 2 + 2 and provide a simple, relatable example to help the student understand and remember the concept. Additionally, address their confusion about having 3 or 4 apples to help clarify the difference between the scenarios.

---

**Elaborated Response:**

You mentioned that you might have 3 apples or 4 apples and you're a bit confused. Let's break it down step-by-step.

For 

In [97]:
adaptive_prompt2 = final_chain.invoke(
    {
        
        'messages': messages,
        'response':"maybe i have 3 apple or 4 i'm bit confused. so 2+2=4",
        
    }
)
print(adaptive_prompt2)

sentiment='partially_correct'
Based on the context and the need to guide the student effectively, here is an adaptive prompt for your main model:

---

**Adaptive Prompt:**

You are an experienced math tutor helping a student who is partially correct but still a bit confused. The student mentioned they might have 3 or 4 apples and is unsure, but correctly identified that 2 + 2 = 4. Now, the student is presented with a new problem: 2 + 2 = 0. Guide the student through this problem by first reinforcing their understanding of basic addition and then addressing the new equation. Use clear and simple language to help them understand why 2 + 2 does not equal 0 and encourage them to think through the logic step-by-step.

---

This prompt will help the main model generate a response that is both instructive and supportive, guiding the student towards the correct understanding.


## get structure adaptive prompt

In [98]:
class AdaptivePrompt(BaseModel):
    adaptive_prompt: str = Field(description="The final adaptive prompt to guide the main model.")


In [104]:
adaptive_parser = PydanticOutputParser(pydantic_object=AdaptivePrompt)

prompt_partially_correct = ChatPromptTemplate.from_messages(
    [
        ("system", 
         """
         Your task is to generate one adaptive prompt only. This prompt will be used as the system prompt for the main model. 
         Think of yourself as a supervisor providing the best instruction for the AI tutor.

         You can analyze the previous chat history and the student’s latest response.

         Student gave a response which is partially correct: {response}

         Return your result in the following format:
         {format_instructions}
         """
         ),
        MessagesPlaceholder(variable_name="messages")
    ]
).partial(format_instructions=adaptive_parser.get_format_instructions())



In [113]:
branch_chain = RunnableBranch(
    (lambda x: x["sentiment"] == 'correct', prompt_correct | model | adaptive_parser),
    (lambda x: x["sentiment"] == 'partially_correct', prompt_partially_correct | model | adaptive_parser),
    (lambda x: x["sentiment"] == 'incorrect', prompt_partially_correct | model | adaptive_parser),
    RunnableLambda(lambda x: {"adaptive_prompt": "Couldn't determine sentiment."})
)

In [114]:
from langchain_core.runnables import RunnableLambda

# Wrap classifier_chain to return full input + parsed sentiment
def enrich_with_sentiment(input: dict) -> dict:
    sentiment_result = classifier_chain.invoke(input)
    print(sentiment_result)
    return {
        **input,  # includes 'messages' and 'response'
        'sentiment': sentiment_result.sentiment
    }

enriched_chain = RunnableLambda(enrich_with_sentiment)

In [115]:
final_chain = enriched_chain | branch_chain

In [116]:
adaptive_prompt3 = final_chain.invoke(
    {
        
        'messages': messages,
        'response':"maybe i have 3 apple or 4 i'm bit confused. so 2+2=4",
        
    }
)
print(adaptive_prompt3)

sentiment='partially_correct'
adaptive_prompt='Guide the student by asking clarifying questions and providing examples to help them understand the concept of addition. Address their confusion about the number of apples and reinforce that 2+2 equals 4. Use relatable examples to make the concept clearer.'
